# Initial loading of the image and set up

In [10]:
import cv2
import numpy as np
from collections import Counter
from matplotlib import pyplot as plt
import ipywidgets as widgets
from IPython.display import display

img = "images/example.JPG"



# Auto Preprocessing Step 

In [17]:
def remove_most_common_color_background(image, tolerance=30):
    # Reshape the image to a list of pixels
    pixels = np.float32(image.reshape(-1, 3))
    
    # Use k-means to cluster the pixel colors
    n_colors = 5
    _, labels, palette = cv2.kmeans(pixels, n_colors, None, 
                                    (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2), 
                                    10, cv2.KMEANS_RANDOM_CENTERS)
    
    # Count each color occurrence
    counts = Counter(labels.flatten())
    
    # Get the most common color
    dominant_color = palette[np.argmax(counts)]
    
    # Create a mask for colors close to the dominant color
    lower_bound = np.maximum(dominant_color - tolerance, 0)
    upper_bound = np.minimum(dominant_color + tolerance, 255)
    mask = cv2.inRange(image, lower_bound, upper_bound)
    
    # Invert the mask to keep everything except the background
    mask_inv = cv2.bitwise_not(mask)
    
    # Remove the background
    result = cv2.bitwise_and(image, image, mask=mask_inv)

    # Optional: Replace the background with white
    white_background = np.full(image.shape, 255, dtype=np.uint8)
    final_image = np.where(result == 0, white_background, result)
    
    return final_image



# Load the image once to avoid reloading it with every update
image_path = img
original_image = cv2.imread(image_path)

def update_preprocessing(tolerance):
    preprocessed_image = remove_most_common_color_background(original_image, tolerance)
    
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(preprocessed_image, cv2.COLOR_BGR2RGB))
    plt.title('Preprocessed Image')
    plt.axis('off')
    
    plt.show()


# Adjusting Preprocessing

In [18]:


# Create a slider widget for tolerance
tolerance_slider = widgets.IntSlider(value=30, min=0, max=100, step=1, description='Tolerance:')
# Display the interactive widget
interactive_plot = widgets.interactive(update_preprocessing, tolerance=tolerance_slider)

display(interactive_plot)

interactive(children=(IntSlider(value=30, description='Tolerance:'), Output()), _dom_classes=('widget-interact…